<a href="https://colab.research.google.com/github/dodo838383/meuportfolio/blob/master/spark_digital_house.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**O que é Spark?**



O Spark é uma plataforma de computação em cluster. Com ele conseguimos distribuir os dados e as tarefas em clusters com vários nós. Entendam cluster como um conjunto de servidores, computadores. E os nós seriam cada computador   individualmente.

E o Spark permite que você distribua os seus dados e as tarefas por esses vários nós do cluster. Ele é uma estrutura de processamento paralelo, que dá também suporte a processamento e memória.

Uma aplicação do Spark pode carregar e armazenar dados em cache de memória, e lê essas informações quantas diversas vezes forem necessárias, diversas vezes.
   
A computação em memória é muito mais rápida do que aplicativos que são baseados em disco, como o Hadoop, por exemplo, que compartilha os dados por meio do seu sistema de arquivos.

E essa característica faz com que o Spark apresente uma performance muito superior ao Hadoop. Em alguns casos chega a ser 100 vezes mais rápido do que o Hadoop, que é bastante coisa.

**Utilizando o Spark no Google Colab**

Para facilitar o desenvolvimento do nosso projeto vamos utilizar o Google Colab como ferramenta e para configurar o PySpark basta executar os comandos abaixo na própria célula do seu notebook

In [348]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark   

In [349]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"   

In [350]:
import findspark
findspark.init()

**Carregamento de Dados**

## [SparkSession](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.SparkSession.html)

O ponto de entrada para programar o Spark com a API Dataset e DataFrame.

Uma SparkSession pode ser utilizada para criar DataFrames, registrar DataFrames como tabelas, executar consultas SQL em tabelas, armazenar em cache e ler arquivos parquet. Para criar uma SparkSession, use o seguinte padrão de construtor:

In [351]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .getOrCreate()

In [352]:
spark

## DataFrames com Spark


### Interfaces Spark

Existem três interfaces principais do Apache Spark que você deve conhecer: Resilient Distributed Dataset, DataFrame e Dataset.

- **Resilient Distributed Dataset**: A primeira abstração do Apache Spark foi o Resilient Distributed Dataset (RDD). É uma interface para uma sequência de objetos de dados que consiste em um ou mais tipos localizados em uma coleção de máquinas (um cluster). Os RDDs podem ser criados de várias maneiras e são a API de “nível mais baixo” disponível. Embora esta seja a estrutura de dados original do Apache Spark, você deve se concentrar na API DataFrame, que é um superconjunto da funcionalidade RDD. A API RDD está disponível nas linguagens Java, Python e Scala.

- **DataFrame**: Trata-se de um conceito similar ao DataFrame que você pode estar familiarizado como o pacote pandas do Python e a linguagem R . A API DataFrame está disponível nas linguagens Java, Python, R e Scala.

- **Dataset**: uma combinação de DataFrame e RDD. Ele fornece a interface digitada que está disponível em RDDs enquanto fornece a conveniência do DataFrame. A API Dataset está disponível nas linguagens Java e Scala.

Em muitos cenários, especialmente com as otimizações de desempenho incorporadas em DataFrames e Datasets, não será necessário trabalhar com RDDs. Mas é importante entender a abstração RDD porque:

- O RDD é a infraestrutura subjacente que permite que o Spark seja executado com tanta rapidez e forneça a linhagem de dados.

- Se você estiver mergulhando em componentes mais avançados do Spark, pode ser necessário usar RDDs.

- As visualizações na Spark UI fazem referência a RDDs.

In [353]:
data = [('Zeca','35'), ('Eva', '29')]
colNames = ['Nome', 'Idade']
df = spark.createDataFrame(data,colNames)
df

DataFrame[Nome: string, Idade: string]

In [354]:
df.show()

+----+-----+
|Nome|Idade|
+----+-----+
|Zeca|   35|
| Eva|   29|
+----+-----+



In [355]:
df.toPandas()

,Nome,Idade
0,Zeca,35
1,Eva,29


## Projeto

Nosso projeto consiste em ler, manipular, tratar e salvar um conjunto de dados volumosos utilizando como ferramenta o Spark.

## Carregamento de dados

### Dados Públicos CNPJ
#### Receita Federal

> [Empresas](https://caelum-online-public.s3.amazonaws.com/2273-introducao-spark/01/empresas.zip)
> 
> [Estabelecimentos](https://caelum-online-public.s3.amazonaws.com/2273-introducao-spark/01/estabelecimentos.zip)
> 
> [Sócios](https://caelum-online-public.s3.amazonaws.com/2273-introducao-spark/01/socios.zip)

[Fonte original dos dados](https://www.gov.br/receitafederal/pt-br/assuntos/orientacao-tributaria/cadastros/consultas/dados-publicos-cnpj)

---
[property SparkSession.read](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.SparkSession.read.html)

[DataFrameReader.csv(*args)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrameReader.csv.html)

### Montando nosso drive

In [356]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [357]:
import zipfile    

In [358]:
zipfile.ZipFile('/content/drive/MyDrive/spark/empresas.zip','r').extractall('/content/drive/MyDrive/spark/empresas')
zipfile.ZipFile('/content/drive/MyDrive/spark/estabelecimentos.zip','r').extractall('/content/drive/MyDrive/spark/estabelecimentos')
zipfile.ZipFile('/content/drive/MyDrive/spark/socios.zip','r').extractall('/content/drive/MyDrive/spark/socios')


In [359]:
path = '/content/drive/MyDrive/spark/empresas/empresas'
empresas = spark.read.csv(path , sep =';' , inferSchema= True)

In [360]:
path = '/content/drive/MyDrive/spark/estabelecimentos/estabelecimentos'
estabelecimentos = spark.read.csv(path , sep =';' , inferSchema= True)

In [361]:
path = '/content/drive/MyDrive/spark/socios/socios'
socios = spark.read.csv(path , sep =';' , inferSchema= True)

In [362]:
empresas.count()

4585679

# Manipulando os Dados

## Operações básicas

### Renomeando as colunas do DataFrame

In [363]:
empresas.limit(5).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,"0,00",1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,"0,00",5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,"0,00",5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,"0,00",5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,"4000,00",1,None


In [364]:
empresasColNames = ['cnpj_basico', 'razao_social_nome_empresarial', 'natureza_juridica', 'qualificacao_do_responsavel', 'capital_social_da_empresa', 'porte_da_empresa', 'ente_federativo_responsavel']

In [365]:
for item in enumerate(empresasColNames):
  print(item)

(0, 'cnpj_basico')
(1, 'razao_social_nome_empresarial')
(2, 'natureza_juridica')
(3, 'qualificacao_do_responsavel')
(4, 'capital_social_da_empresa')
(5, 'porte_da_empresa')
(6, 'ente_federativo_responsavel')


In [366]:
for index, colName in enumerate(empresasColNames):
  empresas = empresas.withColumnRenamed(f"_c{index}",colName)

empresas.columns

['cnpj_basico',
 'razao_social_nome_empresarial',
 'natureza_juridica',
 'qualificacao_do_responsavel',
 'capital_social_da_empresa',
 'porte_da_empresa',
 'ente_federativo_responsavel']

In [367]:
empresas.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,"0,00",1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,"0,00",5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,"0,00",5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,"0,00",5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,"4000,00",1,None


In [368]:
estabelecimentos.limit(5).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c20,_c21,_c22,_c23,_c24,_c25,_c26,_c27,_c28,_c29
0,1879,1,96,1,PIRAMIDE M. C.,8,20011029,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
1,2818,1,43,1,None,8,20081231,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
2,3110,1,7,1,None,8,19971231,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
3,3733,1,80,1,None,8,20081231,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
4,4628,3,27,2,EMBROIDERY & GIFT,8,19980429,1,None,NaN,...,7075,None,None,None,None,NaN,None,None,None,NaN


In [369]:
estabsColNames = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial', 'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', 'nome_da_cidade_no_exterior', 'pais', 'data_de_inicio_atividade', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_de_logradouro', 'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2', 'ddd_do_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_da_situacao_especial']

In [370]:
for index, colName in enumerate(estabsColNames):
  estabelecimentos = estabelecimentos.withColumnRenamed(f"_c{index}",colName)

estabelecimentos.columns

['cnpj_basico',
 'cnpj_ordem',
 'cnpj_dv',
 'identificador_matriz_filial',
 'nome_fantasia',
 'situacao_cadastral',
 'data_situacao_cadastral',
 'motivo_situacao_cadastral',
 'nome_da_cidade_no_exterior',
 'pais',
 'data_de_inicio_atividade',
 'cnae_fiscal_principal',
 'cnae_fiscal_secundaria',
 'tipo_de_logradouro',
 'logradouro',
 'numero',
 'complemento',
 'bairro',
 'cep',
 'uf',
 'municipio',
 'ddd_1',
 'telefone_1',
 'ddd_2',
 'telefone_2',
 'ddd_do_fax',
 'fax',
 'correio_eletronico',
 'situacao_especial',
 'data_da_situacao_especial']

In [371]:
estabelecimentos.limit(5).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dv,identificador_matriz_filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_da_cidade_no_exterior,pais,...,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_do_fax,fax,correio_eletronico,situacao_especial,data_da_situacao_especial
0,1879,1,96,1,PIRAMIDE M. C.,8,20011029,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
1,2818,1,43,1,None,8,20081231,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
2,3110,1,7,1,None,8,19971231,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
3,3733,1,80,1,None,8,20081231,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
4,4628,3,27,2,EMBROIDERY & GIFT,8,19980429,1,None,NaN,...,7075,None,None,None,None,NaN,None,None,None,NaN


In [372]:
socios.limit(5).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,19940725,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,19940725,NaN,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,19940516,NaN,***000000**,None,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,19940516,NaN,***000000**,None,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,19940609,NaN,***000000**,None,0,8


In [373]:
sociosColNames = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social', 'cnpj_ou_cpf_do_socio', 'qualificacao_do_socio', 'data_de_entrada_sociedade', 'pais', 'representante_legal', 'nome_do_representante', 'qualificacao_do_representante_legal', 'faixa_etaria']

In [374]:
for index, colName in enumerate(sociosColNames):
  socios = socios.withColumnRenamed(f"_c{index}",colName)

socios.columns

['cnpj_basico',
 'identificador_de_socio',
 'nome_do_socio_ou_razao_social',
 'cnpj_ou_cpf_do_socio',
 'qualificacao_do_socio',
 'data_de_entrada_sociedade',
 'pais',
 'representante_legal',
 'nome_do_representante',
 'qualificacao_do_representante_legal',
 'faixa_etaria']

In [375]:
socios.limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,19940725,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,19940725,NaN,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,19940516,NaN,***000000**,None,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,19940516,NaN,***000000**,None,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,19940609,NaN,***000000**,None,0,8


## Analisando os dados

[Data Types](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#data-types)

In [376]:
empresas.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,"0,00",1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,"0,00",5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,"0,00",5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,"0,00",5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,"4000,00",1,None


In [377]:
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: string (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



In [378]:
socios.limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,19940725,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,19940725,NaN,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,19940516,NaN,***000000**,None,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,19940516,NaN,***000000**,None,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,19940609,NaN,***000000**,None,0,8


In [379]:
socios.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: integer (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



In [380]:
estabelecimentos.limit(5).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dv,identificador_matriz_filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_da_cidade_no_exterior,pais,...,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_do_fax,fax,correio_eletronico,situacao_especial,data_da_situacao_especial
0,1879,1,96,1,PIRAMIDE M. C.,8,20011029,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
1,2818,1,43,1,None,8,20081231,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
2,3110,1,7,1,None,8,19971231,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
3,3733,1,80,1,None,8,20081231,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
4,4628,3,27,2,EMBROIDERY & GIFT,8,19980429,1,None,NaN,...,7075,None,None,None,None,NaN,None,None,None,NaN


In [381]:
estabelecimentos.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: integer (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- data_de_inicio_atividade: integer (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: str

## Modificando os tipos de dados

[Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions)

[withColumn](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.withColumn.html)

### Convertendo String ➔ Double

#### `StringType ➔ DoubleType`

In [382]:
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

In [383]:
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: string (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



In [384]:
empresas.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,"0,00",1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,"0,00",5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,"0,00",5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,"0,00",5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,"4000,00",1,None


In [385]:
empresas = empresas.withColumn('capital_social_da_empresa', f.regexp_replace('capital_social_da_empresa', ',', '.'))
empresas.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,0.00,1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,0.00,5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,0.00,5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,0.00,5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,4000.00,1,None


In [386]:
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: string (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



In [387]:
empresas = empresas.withColumn('capital_social_da_empresa',empresas['capital_social_da_empresa'].cast(DoubleType()))
empresas.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,0.0,1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,0.0,5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,0.0,5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,0.0,5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,4000.0,1,None


In [388]:
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: double (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



### Convertendo String ➔ Date

#### `StringType ➔ DateType`

[Datetime Patterns](https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html)

In [389]:
df = spark.createDataFrame([(20200924,), (20201022,), (20210215,)], ['data'])
df.toPandas()

,data
0,20200924
1,20201022
2,20210215


In [390]:
df.printSchema()

root
 |-- data: long (nullable = true)



In [391]:
df = df.withColumn("data", f.to_date(df.data.cast(StringType()),'yyyyMMdd'))
df.printSchema()

root
 |-- data: date (nullable = true)



In [392]:
df.toPandas()

,data
0,2020-09-24
1,2020-10-22
2,2021-02-15


In [393]:
estabelecimentos.limit(5).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dv,identificador_matriz_filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_da_cidade_no_exterior,pais,...,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_do_fax,fax,correio_eletronico,situacao_especial,data_da_situacao_especial
0,1879,1,96,1,PIRAMIDE M. C.,8,20011029,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
1,2818,1,43,1,None,8,20081231,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
2,3110,1,7,1,None,8,19971231,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
3,3733,1,80,1,None,8,20081231,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
4,4628,3,27,2,EMBROIDERY & GIFT,8,19980429,1,None,NaN,...,7075,None,None,None,None,NaN,None,None,None,NaN


In [394]:
estabelecimentos.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: integer (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- data_de_inicio_atividade: integer (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: str

In [395]:
estabelecimentos = estabelecimentos\
.withColumn(
            "data_situacao_cadastral", 
            f.to_date(estabelecimentos.data_situacao_cadastral.cast(StringType()),'yyyyMMdd')
            ) \
.withColumn(
            "data_de_inicio_atividade", 
            f.to_date(estabelecimentos.data_de_inicio_atividade.cast(StringType()),'yyyyMMdd') 
            )\
.withColumn(
            "data_da_situacao_especial", 
            f.to_date(estabelecimentos.data_da_situacao_especial.cast(StringType()),'yyyyMMdd') 
            )

In [396]:
estabelecimentos.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- data_de_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: string (n

In [397]:
estabelecimentos.limit(5).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dv,identificador_matriz_filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_da_cidade_no_exterior,pais,...,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_do_fax,fax,correio_eletronico,situacao_especial,data_da_situacao_especial
0,1879,1,96,1,PIRAMIDE M. C.,8,2001-10-29,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,None
1,2818,1,43,1,None,8,2008-12-31,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,None
2,3110,1,7,1,None,8,1997-12-31,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,None
3,3733,1,80,1,None,8,2008-12-31,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,None
4,4628,3,27,2,EMBROIDERY & GIFT,8,1998-04-29,1,None,NaN,...,7075,None,None,None,None,NaN,None,None,None,None


In [398]:
socios.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: integer (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



In [399]:
socios = socios\
.withColumn(
            "data_de_entrada_sociedade", 
            f.to_date(socios.data_de_entrada_sociedade.cast(StringType()),'yyyyMMdd')
            )    

In [400]:
socios.limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,1994-07-25,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,1994-07-25,NaN,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,1994-05-16,NaN,***000000**,None,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,1994-05-16,NaN,***000000**,None,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,1994-06-09,NaN,***000000**,None,0,8


In [401]:
socios.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: date (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



# Seleções e consultas
---

## Selecionando informações
 
[DataFrame.select(*cols)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.select.html)

In [402]:
empresas\
    .select('*')\
    .show(5, False)

+-----------+--------------------------------------------------------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial                                                               |natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+--------------------------------------------------------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|306        |FRANCAMAR REFRIGERACAO TECNICA S/C LTDA                                                     |2240             |49                         |0.0                      |1               |null                       |
|1355       |BRASILEIRO & OLIVEIRA LTDA                                                                 

In [403]:
empresas\
    .select('natureza_juridica','porte_da_empresa','capital_social_da_empresa')\
    .show(5, False)

+-----------------+----------------+-------------------------+
|natureza_juridica|porte_da_empresa|capital_social_da_empresa|
+-----------------+----------------+-------------------------+
|2240             |1               |0.0                      |
|2062             |5               |0.0                      |
|3034             |5               |0.0                      |
|2135             |5               |0.0                      |
|2062             |1               |4000.0                   |
+-----------------+----------------+-------------------------+
only showing top 5 rows



In [404]:
socios\
    .select('nome_do_socio_ou_razao_social','faixa_etaria',f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .show(5, False)

+-------------------------------+------------+--------------+
|nome_do_socio_ou_razao_social  |faixa_etaria|ano_de_entrada|
+-------------------------------+------------+--------------+
|LILIANA PATRICIA GUASTAVINO    |7           |1994          |
|CRISTINA HUNDERTMARK           |7           |1994          |
|CELSO EDUARDO DE CASTRO STEPHAN|8           |1994          |
|EDUARDO BERRINGER STEPHAN      |5           |1994          |
|HANNE MAHFOUD FADEL            |8           |1994          |
+-------------------------------+------------+--------------+
only showing top 5 rows



## Identificando valores nulos

In [405]:
df = spark.createDataFrame([(1,), (2,), (3,), (None,)], ['data'])
df.toPandas()

,data
0,1.0
1,2.0
2,3.0
3,NaN


In [406]:
df.show()

+----+
|data|
+----+
|   1|
|   2|
|   3|
|null|
+----+



In [407]:
df = spark.createDataFrame([(1.,), (2.,), (3.,), (float('nan'),)], ['data'])
df.toPandas()

,data
0,1.0
1,2.0
2,3.0
3,NaN


In [408]:
df.show()

+----+
|data|
+----+
| 1.0|
| 2.0|
| 3.0|
| NaN|
+----+



In [409]:
df = spark.createDataFrame([('1',), ('2',), ('3',), (None,)], ['data'])
df.toPandas()

,data
0,1
1,2
2,3
3,None


In [410]:
df.show()

+----+
|data|
+----+
|   1|
|   2|
|   3|
|null|
+----+



In [411]:
socios.limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,1994-07-25,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,1994-07-25,NaN,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,1994-05-16,NaN,***000000**,None,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,1994-05-16,NaN,***000000**,None,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,1994-06-09,NaN,***000000**,None,0,8


In [412]:
socios.limit(5).show()

+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|cnpj_basico|identificador_de_socio|nome_do_socio_ou_razao_social|cnpj_ou_cpf_do_socio|qualificacao_do_socio|data_de_entrada_sociedade|pais|representante_legal|nome_do_representante|qualificacao_do_representante_legal|faixa_etaria|
+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|        411|                     2|         LILIANA PATRICIA ...|         ***678188**|                   22|               1994-07-25|null|        ***000000**|                 null|                                  0|           7|
|        411|                     2|         CRISTINA HUNDERTMARK|      

In [413]:
socios.select([f.count(f.when(f.isnull(c),1)).alias(c) for  c in socios.columns]).show()

+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+-------+-------------------+---------------------+-----------------------------------+------------+
|cnpj_basico|identificador_de_socio|nome_do_socio_ou_razao_social|cnpj_ou_cpf_do_socio|qualificacao_do_socio|data_de_entrada_sociedade|   pais|representante_legal|nome_do_representante|qualificacao_do_representante_legal|faixa_etaria|
+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+-------+-------------------+---------------------+-----------------------------------+------------+
|          0|                     0|                          208|                1234|                    0|                        1|2038255|                  0|              1995432|                                  0|           0|
+-----------+----------------------+------------------------

In [414]:
socios.limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,1994-07-25,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,1994-07-25,NaN,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,1994-05-16,NaN,***000000**,None,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,1994-05-16,NaN,***000000**,None,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,1994-06-09,NaN,***000000**,None,0,8


In [415]:
socios.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: date (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



In [416]:
socios.na.fill(0).limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,1994-07-25,0,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,1994-07-25,0,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,1994-05-16,0,***000000**,None,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,1994-05-16,0,***000000**,None,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,1994-06-09,0,***000000**,None,0,8


In [417]:
socios.na.fill('-').limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,1994-07-25,NaN,***000000**,-,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,1994-07-25,NaN,***000000**,-,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,1994-05-16,NaN,***000000**,-,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,1994-05-16,NaN,***000000**,-,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,1994-06-09,NaN,***000000**,-,0,8


In [418]:
socios = socios.na.fill(0)
socios = socios.na.fill('-')

In [419]:
socios.limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,1994-07-25,0,***000000**,-,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,1994-07-25,0,***000000**,-,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,1994-05-16,0,***000000**,-,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,1994-05-16,0,***000000**,-,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,1994-06-09,0,***000000**,-,0,8


## Ordenando os dados

[DataFrame.orderBy(*cols, **kwargs)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.orderBy.html)

In [420]:
socios\
    .select('nome_do_socio_ou_razao_social','faixa_etaria',f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .orderBy('ano_de_entrada',ascending = False)\
    .show(5, False)

+-----------------------------+------------+--------------+
|nome_do_socio_ou_razao_social|faixa_etaria|ano_de_entrada|
+-----------------------------+------------+--------------+
|JOSE HUMBERTO PAIVA          |6           |2021          |
|BENILDES BARBOSA RODRIGUES   |8           |2021          |
|MARCELO MOCELIN              |5           |2021          |
|ROBERTA BENELLI              |4           |2021          |
|EDUARDO DE ANDRADE PEIXOTO   |5           |2021          |
+-----------------------------+------------+--------------+
only showing top 5 rows



In [421]:
socios\
    .select('nome_do_socio_ou_razao_social','faixa_etaria',f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .orderBy(['ano_de_entrada','faixa_etaria'],ascending = [False,False])\
    .show(10, False)

+---------------------------------+------------+--------------+
|nome_do_socio_ou_razao_social    |faixa_etaria|ano_de_entrada|
+---------------------------------+------------+--------------+
|ANTONIO TAVARES DE ANDRADE       |9           |2021          |
|ANNA MARIA TELLES FERREIRA SANTOS|9           |2021          |
|ANTONIA DE SOUSA VIEIRA          |9           |2021          |
|AURA MARIA DE ANDRADE            |9           |2021          |
|SONIA MARQUES SAMAJA             |9           |2021          |
|CARLOS ERANE DE AGUIAR           |9           |2021          |
|MATILDE CONCEICAO DE JESUS       |9           |2021          |
|MANUEL TAVARES DE SOUSA          |9           |2021          |
|ALBERTO DE BANDOS MENDES         |9           |2021          |
|EMERSON AZEVEDO                  |9           |2021          |
+---------------------------------+------------+--------------+
only showing top 10 rows



## Filtrando os dados

[DataFrame.where(condition)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.where.html) ou [DataFrame.filter(condition)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.filter.html)

In [422]:
empresas\
    .where("capital_social_da_empresa =50")\
    .show(5, False)

+-----------+------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial       |natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|17350147   |ERIK MARCELO DOS SANTOS 42107848858 |2135             |50                         |50.0                     |1               |null                       |
|17833214   |ALEXANDRE MACHADO LIMA 73750123772  |2135             |50                         |50.0                     |1               |null                       |
|20860830   |YASMIN MOURA DA FONSECA 13457709793 |2135             |50                         |50.0                     |1               |null                 

In [423]:
socios\
    .select("nome_do_socio_ou_razao_social")\
    .filter(socios.nome_do_socio_ou_razao_social.startswith("CLAUDIO"))\
    .filter(socios.nome_do_socio_ou_razao_social.endswith("JOSE"))\
    .limit(10)\
    .toPandas()

,nome_do_socio_ou_razao_social
0,CLAUDIO JOSE
1,CLAUDIO LUIZ BRANDAO JOSE
2,CLAUDIO ANTONIO JOSE
3,CLAUDIO NADIRO SALOMAO JOSE


## O comando LIKE

[Column.like(other)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.Column.like.html)

In [424]:
df = spark.createDataFrame([('RESTAURANTE DO RUI',), ('Juca restaurantes ltda',), ('Joca Restaurante',)], ['data'])
df.toPandas()

,data
0,RESTAURANTE DO RUI
1,Juca restaurantes ltda
2,Joca Restaurante


In [425]:
df\
    .where(f.upper(df.data).like('%RESTAURANTE%'))\
    .show(truncate = False)

+----------------------+
|data                  |
+----------------------+
|RESTAURANTE DO RUI    |
|Juca restaurantes ltda|
|Joca Restaurante      |
+----------------------+



In [426]:
#Caso retire o percentual da esquerda ele trará somente os resultados de quem começa com restaurante
df\
    .where(f.upper(df.data).like('RESTAURANTE%'))\
    .show(truncate = False)

+------------------+
|data              |
+------------------+
|RESTAURANTE DO RUI|
+------------------+



In [427]:
#da mesma forma retirando o percentual da direita ele trará somente o resultado de quem termina com a palavra procurada.
df\
    .where(f.upper(df.data).like('%RESTAURANTE'))\
    .show(truncate = False)

+----------------+
|data            |
+----------------+
|Joca Restaurante|
+----------------+



In [428]:
empresas\
      .select('razao_social_nome_empresarial','natureza_juridica','porte_da_empresa','capital_social_da_empresa')\
      .filter(f.upper(empresas.razao_social_nome_empresarial).like('%RESTAURANTE'))\
      .show(15,False)

+----------------------------------------------------+-----------------+----------------+-------------------------+
|razao_social_nome_empresarial                       |natureza_juridica|porte_da_empresa|capital_social_da_empresa|
+----------------------------------------------------+-----------------+----------------+-------------------------+
|MARIA ROZA DOS SANTOS- BAR E RESTAURANTE            |2135             |5               |0.0                      |
|ELIANIA A. CUSTODIO RESTAURANTE                     |2135             |1               |0.0                      |
|R. A. D. ABRIL RESTAURANTE                          |2135             |1               |0.0                      |
|R. DA S. ARAUJO - RESTAURANTE                       |2135             |5               |0.0                      |
|A C M COUTINHO-LANCHONETE E RESTAURANTE             |2135             |1               |0.0                      |
|REGINA DA S SANTOS RESTAURANTE                      |2135             |

# Agregações e Junções
---

[DataFrame.groupBy(*cols)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.groupBy.html)

[DataFrame.agg(*exprs)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.agg.html)

[DataFrame.summary(*statistics)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.summary.html)

> Funções:
[approx_count_distinct](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.approx_count_distinct.html) | 
[avg](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.avg.html) | 
[collect_list](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.collect_list.html) | 
[collect_set](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.collect_set.html) | 
[countDistinct](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.countDistinct.html) | 
[count](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.count.html) | 
[grouping](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.grouping.html) | 
[first](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.first.html) | 
[last](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.last.html) | 
[kurtosis](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.kurtosis.html) | 
[max](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.max.html) | 
[min](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.min.html) | 
[mean](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.mean.html) | 
[skewness](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.skewness.html) | 
[stddev ou stddev_samp](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.stddev.html) | 
[stddev_pop](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.stddev_pop.html) | 
[sum](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.sum.html) | 
[sumDistinct](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.sumDistinct.html) | 
[variance ou var_samp](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.variance.html) | 
[var_pop](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.var_pop.html)

## Sumarizando os dados

In [429]:
socios\
    .select(f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .where("ano_de_entrada >= 2010")\
    .groupBy('ano_de_entrada')\
    .count()\
    .orderBy('ano_de_entrada', ascending = True)\
    .show()

+--------------+------+
|ano_de_entrada| count|
+--------------+------+
|          2010| 79337|
|          2011| 83906|
|          2012| 80101|
|          2013| 83919|
|          2014| 80590|
|          2015| 80906|
|          2016| 81587|
|          2017| 90221|
|          2018| 99935|
|          2019|118248|
|          2020|125927|
|          2021| 56316|
+--------------+------+



In [430]:
empresas\
    .select('cnpj_basico','porte_da_empresa','capital_social_da_empresa')\
    .groupBy('porte_da_empresa')\
    .agg(
        f.avg("capital_social_da_empresa").alias("capital_social_medio"),
        f.count("cnpj_basico").alias("frequencia")
    )\
    .orderBy('porte_da_empresa', ascending = True)\
    .show()

+----------------+--------------------+----------+
|porte_da_empresa|capital_social_medio|frequencia|
+----------------+--------------------+----------+
|            null|    8.35421888053467|      5985|
|               1|  339994.53313506936|   3129043|
|               3|  2601001.7677092673|    115151|
|               5|   708660.4208249798|   1335500|
+----------------+--------------------+----------+



In [431]:
#Equivalente Describe Pandas
empresas\
    .select("capital_social_da_empresa")\
    .summary()\
    .show()

#stddev = desvio padrão

+-------+-------------------------+
|summary|capital_social_da_empresa|
+-------+-------------------------+
|  count|                  4585679|
|   mean|        503694.5478542675|
| stddev|     2.1118691490537405E8|
|    min|                      0.0|
|    25%|                      0.0|
|    50%|                   1000.0|
|    75%|                   7000.0|
|    max|         3.22014670262E11|
+-------+-------------------------+



## Juntando DataFrames - Joins

[DataFrame.join(*args)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.join.html)

In [432]:
produtos = spark.createDataFrame(
    [
     ('1','Bebidas','Água mineral'),
     ('2','Limpeza','Sabão em pó'),
     ('3','Frios','Queijo'),
     ('4','Bebidas','Refrigerantes'),
     ('5','Pet','Ração para cães')     
     ],
      ['id','cat','prod']
)

impostos = spark.createDataFrame(
    [
     ('Bebidas',0.15),
     ('Limpeza',0.05),
     ('Frios',0.065),
     ('Carnes',0.08)
    ],
    ['cat','tax']
)

In [433]:
produtos.toPandas()

,id,cat,prod
0,1,Bebidas,Água mineral
1,2,Limpeza,Sabão em pó
2,3,Frios,Queijo
3,4,Bebidas,Refrigerantes
4,5,Pet,Ração para cães


In [434]:
impostos.toPandas()

,cat,tax
0,Bebidas,0.150
1,Limpeza,0.050
2,Frios,0.065
3,Carnes,0.080


In [435]:
produtos.join(impostos,'cat',how = 'inner')\
    .sort('id')\
    .show()

+-------+---+-------------+-----+
|    cat| id|         prod|  tax|
+-------+---+-------------+-----+
|Bebidas|  1| Água mineral| 0.15|
|Limpeza|  2|  Sabão em pó| 0.05|
|  Frios|  3|       Queijo|0.065|
|Bebidas|  4|Refrigerantes| 0.15|
+-------+---+-------------+-----+



In [436]:
produtos.join(impostos,'cat',how = 'left')\
    .sort('id')\
    .show()

+-------+---+---------------+-----+
|    cat| id|           prod|  tax|
+-------+---+---------------+-----+
|Bebidas|  1|   Água mineral| 0.15|
|Limpeza|  2|    Sabão em pó| 0.05|
|  Frios|  3|         Queijo|0.065|
|Bebidas|  4|  Refrigerantes| 0.15|
|    Pet|  5|Ração para cães| null|
+-------+---+---------------+-----+



In [437]:
produtos.join(impostos,'cat',how = 'right')\
    .sort('id')\
    .show()

+-------+----+-------------+-----+
|    cat|  id|         prod|  tax|
+-------+----+-------------+-----+
| Carnes|null|         null| 0.08|
|Bebidas|   1| Água mineral| 0.15|
|Limpeza|   2|  Sabão em pó| 0.05|
|  Frios|   3|       Queijo|0.065|
|Bebidas|   4|Refrigerantes| 0.15|
+-------+----+-------------+-----+



In [438]:
produtos.join(impostos,'cat',how = 'outer')\
    .sort('id')\
    .show()

+-------+----+---------------+-----+
|    cat|  id|           prod|  tax|
+-------+----+---------------+-----+
| Carnes|null|           null| 0.08|
|Bebidas|   1|   Água mineral| 0.15|
|Limpeza|   2|    Sabão em pó| 0.05|
|  Frios|   3|         Queijo|0.065|
|Bebidas|   4|  Refrigerantes| 0.15|
|    Pet|   5|Ração para cães| null|
+-------+----+---------------+-----+



In [439]:
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: double (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



In [440]:
socios.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = false)
 |-- cnpj_ou_cpf_do_socio: string (nullable = false)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: date (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = false)
 |-- nome_do_representante: string (nullable = false)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



In [441]:
estabelecimentos.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- data_de_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: string (n

In [442]:
empresas_join = estabelecimentos.join(empresas,'cnpj_basico',how = 'inner')

In [443]:
empresas_join.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- data_de_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: string (n

In [444]:
empresas_join.show(5,False)

+-----------+----------+-------+---------------------------+--------------------------+------------------+-----------------------+-------------------------+--------------------------+----+------------------------+---------------------+----------------------+------------------+------------------+------+-------------------+----------------+--------+---+---------+-----+----------+-----+----------+----------+----+----------------------------+-----------------+-------------------------+---------------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|cnpj_ordem|cnpj_dv|identificador_matriz_filial|nome_fantasia             |situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_da_cidade_no_exterior|pais|data_de_inicio_atividade|cnae_fiscal_principal|cnae_fiscal_secundaria|tipo_de_logradouro|logradouro        |numero|complemento        |bairro          |cep 

In [445]:
freq = empresas_join\
      .select(
          'cnpj_basico',
          f.year('data_de_inicio_atividade').alias('data_de_inicio')
      )\
      .where('data_de_inicio >=2010')\
      .groupBy('data_de_inicio')\
      .agg(f.count("cnpj_basico").alias("frequencia"))\
      .orderBy('data_de_inicio', ascending = True)

In [446]:
freq.toPandas()

,data_de_inicio,frequencia
0,2010,154159
1,2011,172677
2,2012,232480
3,2013,198424
4,2014,202276
5,2015,212523
6,2016,265417
7,2017,237292
8,2018,275435
9,2019,325922


In [447]:
freq.union(
    freq.select(
        f.lit('Total').alias('data_de_inicio'),
        f.sum(freq.frequencia).alias('frequencia')
    )

).show()

+--------------+----------+
|data_de_inicio|frequencia|
+--------------+----------+
|          2010|    154159|
|          2011|    172677|
|          2012|    232480|
|          2013|    198424|
|          2014|    202276|
|          2015|    212523|
|          2016|    265417|
|          2017|    237292|
|          2018|    275435|
|          2019|    325922|
|          2020|    400654|
|          2021|    153275|
|         Total|   2830534|
+--------------+----------+



## SparkSQL

[SparkSession.sql(sqlQuery)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.SparkSession.sql.html)

Para saber mais sobre performance: [Artigo - Spark RDDs vs DataFrames vs SparkSQL](https://community.cloudera.com/t5/Community-Articles/Spark-RDDs-vs-DataFrames-vs-SparkSQL/ta-p/246547)

In [448]:
empresas.createOrReplaceTempView("empresasView")
socios.createOrReplaceTempView("sociosView")
estabelecimentos.createOrReplaceTempView("estabelecimentosView")

In [449]:
spark.sql(" SELECT*\
              FROM empresasView").show(5,False)

+-----------+--------------------------------------------------------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial                                                               |natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+--------------------------------------------------------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|306        |FRANCAMAR REFRIGERACAO TECNICA S/C LTDA                                                     |2240             |49                         |0.0                      |1               |null                       |
|1355       |BRASILEIRO & OLIVEIRA LTDA                                                                 

In [450]:
spark.sql("""

          WITH dados AS (

          SELECT
          YEAR(data_de_entrada_sociedade) AS ano_entrada,
          cnpj_basico 
          FROM sociosView           
          
          ),          
        
          clientes as (
            SELECT
            cast(d.ano_entrada as integer) AS ano_entrada
            , d.cnpj_basico AS cnpj_basico
            , e.uf AS uf
            from dados as d
            LEFT JOIN estabelecimentosView as e ON d.cnpj_basico = e.cnpj_basico
                               
          
          )



          SELECT
          COUNT(DISTINCT(cnpj_basico)) AS total_cnpj,
          ano_entrada
          
          FROM clientes
          where ano_entrada > 2004
          GROUP BY 2       

             
              
              """).show()

+----------+-----------+
|total_cnpj|ano_entrada|
+----------+-----------+
|     41655|       2007|
|     65135|       2018|
|     52994|       2015|
|     40755|       2006|
|     53955|       2013|
|     52087|       2014|
|     75799|       2019|
|     82061|       2020|
|     50910|       2012|
|     47585|       2009|
|     53916|       2016|
|     90616|       2005|
|     49555|       2010|
|     52068|       2011|
|     45039|       2008|
|     59823|       2017|
|     37784|       2021|
+----------+-----------+



In [451]:
spark.sql("""SELECT*
              FROM estabelecimentosView """).show(5,False)

+-----------+----------+-------+---------------------------+-----------------+------------------+-----------------------+-------------------------+--------------------------+----+------------------------+---------------------+----------------------+------------------+----------------------+------+-----------+------------------+-------+---+---------+-----+----------+-----+----------+----------+----+------------------+-----------------+-------------------------+
|cnpj_basico|cnpj_ordem|cnpj_dv|identificador_matriz_filial|nome_fantasia    |situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_da_cidade_no_exterior|pais|data_de_inicio_atividade|cnae_fiscal_principal|cnae_fiscal_secundaria|tipo_de_logradouro|logradouro            |numero|complemento|bairro            |cep    |uf |municipio|ddd_1|telefone_1|ddd_2|telefone_2|ddd_do_fax|fax |correio_eletronico|situacao_especial|data_da_situacao_especial|
+-----------+----------+-------+---------------------------+----------

In [452]:
#cnpj_basico chave
#capital_social_da_empresa 
#nome_do_socio_ou_razao_social - socios 
#data_de_entrada_sociedade - socio

In [453]:
spark.sql("""
  SELECT*
  FROM estabelecimentosView

""").show()


+-----------+----------+-------+---------------------------+--------------------+------------------+-----------------------+-------------------------+--------------------------+----+------------------------+---------------------+----------------------+------------------+--------------------+------+-------------------+------------------+--------+---+---------+-----+----------+-----+----------+----------+--------+--------------------+-----------------+-------------------------+
|cnpj_basico|cnpj_ordem|cnpj_dv|identificador_matriz_filial|       nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_da_cidade_no_exterior|pais|data_de_inicio_atividade|cnae_fiscal_principal|cnae_fiscal_secundaria|tipo_de_logradouro|          logradouro|numero|        complemento|            bairro|     cep| uf|municipio|ddd_1|telefone_1|ddd_2|telefone_2|ddd_do_fax|     fax|  correio_eletronico|situacao_especial|data_da_situacao_especial|
+-----------+----------+-------+------

In [454]:
spark.sql("""SELECT*
              FROM sociosView
              WHERE nome_do_socio_ou_razao_social  LIKE '%a%' 
              and nome_do_socio_ou_razao_social  LIKE '%b%' """).show(5,False)

+-----------+----------------------+-----------------------------------------------------------------------------+--------------------+---------------------+-------------------------+----+-------------------+------------------------+-----------------------------------+------------+
|cnpj_basico|identificador_de_socio|nome_do_socio_ou_razao_social                                                |cnpj_ou_cpf_do_socio|qualificacao_do_socio|data_de_entrada_sociedade|pais|representante_legal|nome_do_representante   |qualificacao_do_representante_legal|faixa_etaria|
+-----------+----------------------+-----------------------------------------------------------------------------+--------------------+---------------------+-------------------------+----+-------------------+------------------------+-----------------------------------+------------+
|20529552   |1                     |BSP Empreendimentos Imobili�rios R2 Ltda.                                    |09437113000123      |22              

# Formas de Armazenamento

## Arquivos CSV

[property DataFrame.write](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.write.html)

[DataFrameWriter.csv(*args)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrameWriter.csv.html)

In [455]:
empresas.write.csv(
    path='/content/drive/MyDrive/curso-spark/empresas/csv',
    mode = 'overwrite' , 
    sep = ';',
    header = True
)

In [456]:
empresas2 = spark.read.csv(
    '/content/drive/MyDrive/curso-spark/empresas/csv',
    sep =';',
    inferSchema = True,
    header = True
)

In [457]:
empresas2.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: double (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



## Arquivos PARQUET

[Apache Parquet](https://parquet.apache.org/)

[DataFrameWriter.parquet(*args)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrameWriter.parquet.html)

In [458]:
empresas.write.parquet(
    path='/content/drive/MyDrive/curso-spark/empresas/parquet',
    mode = 'overwrite' 
)

In [459]:
empresas_parquet = spark.read.parquet(
    '/content/drive/MyDrive/curso-spark/empresas/parquet'    
)

In [460]:
empresas_parquet.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: double (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



## Particionamento dos dados

[DataFrameWriter.partitionBy(*cols)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrameWriter.partitionBy.html)

In [461]:
empresas.coalesce(1).write.csv(
    path='/content/drive/MyDrive/curso-spark/empresas/csv-unico',
    mode = 'overwrite' , 
    sep = ';',
    header = True
)

In [462]:
empresas.write.parquet(
    path = '/content/drive/MyDrive/curso-spark/empresas/parquet-partitionBy',
    mode = 'overwrite',
    partitionBy =  'porte_da_empresa'
)

In [463]:
spark.stop()